In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv(r"C:\Users\hp\OneDrive\Desktop\BostonCrime\crime.csv", encoding='latin-1', on_bad_lines="skip")

In [2]:
data.head(2)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"


### Data Analysis Exercise

In [3]:
data['time_column'] = pd.to_datetime(data['OCCURRED_ON_DATE']).dt.time

In [4]:
def phase_of_the_day(time_obj):
    # Convert time object to string
    time_str = time_obj.strftime('%H:%M:%S')

    # Split the time string into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_str.split(':'))

    # Convert hours to 24-hour format
    if hours < 6:
        hours += 24

    # Define phase of the day
    if 6 <= hours < 11:
        return "Morning"
    elif 11 <= hours < 17:
        return "Noon"
    elif 17 <= hours < 20:
        return "Evening"
    else:
        return "Night"

In [5]:
import tqdm
from tqdm import tqdm
tqdm.pandas()
data['phase_of_day'] = data['time_column'].progress_apply(lambda x: phase_of_the_day(x))

100%|███████████████████████████████████████████████████████████████████████| 319073/319073 [00:04<00:00, 74722.88it/s]


In [6]:
import calendar

def month_number_to_value(month_number):
    # Check if the month number is valid
    if 1 <= month_number <= 12:
        # Get the month value using calendar.month_name
        month_value = calendar.month_name[month_number]
        return month_value
    else:
        return "Invalid month number"

In [7]:
data['MONTH_VALUE'] = data['MONTH'].progress_apply(lambda x: month_number_to_value(x))

100%|███████████████████████████████████████████████████████████████████████| 319073/319073 [00:03<00:00, 93214.78it/s]


In [8]:
data.head(1)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,time_column,phase_of_day,MONTH_VALUE
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",13:00:00,Noon,September


In [9]:
test = data.groupby(['phase_of_day', 'MONTH_VALUE'])['OFFENSE_DESCRIPTION'].count().to_frame().reset_index()
# pivot_df = test.pivot(index='Date', columns='Product', values='Sales')


In [10]:
pivot_table = data.pivot_table(index = ['phase_of_day'], columns='MONTH_VALUE', values='OFFENSE_DESCRIPTION', aggfunc='count')
pivot_table

MONTH_VALUE,April,August,December,February,January,July,June,March,May,November,October,September
phase_of_day,,,,,,,,,,,,
Evening,4415,6392,4185,3946,4243,6410,5573,4535,4925,4391,4672,4966
Morning,4341,6331,4253,3949,4375,6015,5472,4473,4806,4383,4766,4842
Night,7389,10972,7022,6383,7119,10848,9570,6988,7732,6877,7862,8097
Noon,7941,11128,8017,7375,7873,11283,9953,8150,8736,8024,8437,8638


In [11]:
offence_df = pd.read_csv(r"C:\Users\hp\OneDrive\Desktop\BostonCrime\offense_codes.csv", encoding='latin-1', on_bad_lines="skip")

In [12]:
offence_df.head(2)

,CODE,NAME
0,612,LARCENY PURSE SNATCH - NO FORCE
1,613,LARCENY SHOPLIFTING


In [13]:
combine_data = data.merge(offence_df , left_on = 'OFFENSE_CODE', right_on = 'CODE')
combine_data.head(2)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,...,UCR_PART,STREET,Lat,Long,Location,time_column,phase_of_day,MONTH_VALUE,CODE,NAME
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,...,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",13:00:00,Noon,September,619,LARCENY ALL OTHERS
1,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,...,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",13:00:00,Noon,September,619,LARCENY OTHER $200 & OVER


In [14]:
### Now We will do the count of offense per district 
combine_data.groupby('DISTRICT')['NAME'].count().reset_index().rename(columns={"NAME":'occurence_of_offense_per_district'}).assign(
Percent = lambda x: 100*(x['occurence_of_offense_per_district']/x['occurence_of_offense_per_district'].sum()))

,DISTRICT,occurence_of_offense_per_district,Percent
0,A1,66302,11.531665
1,A15,11588,2.015459
2,A7,24488,4.259109
3,B2,89529,15.571452
4,B3,64743,11.260514
5,C11,76824,13.361718
6,C6,42358,7.367172
7,D14,36032,6.266914
8,D4,77130,13.414940
9,E13,31594,5.495029


In [15]:
data.head(1)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,time_column,phase_of_day,MONTH_VALUE
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",13:00:00,Noon,September


In [16]:
def last_incidence_date(group):
    return group.max()

# Apply the custom function to each group
data['date_of_last_incidence_happened'] = data.groupby('DISTRICT')['OCCURRED_ON_DATE'].transform(last_incidence_date)


In [17]:
data['DISTRICT OCCURRED_ON_DATE date_of_last_incidence_happened'.split()]

,DISTRICT,OCCURRED_ON_DATE,date_of_last_incidence_happened
0,D14,2018-09-02 13:00:00,2018-09-03 14:06:00
1,C11,2018-08-21 00:00:00,2018-09-03 21:09:00
2,D4,2018-09-03 19:27:00,2018-09-03 21:16:00
3,D4,2018-09-03 21:16:00,2018-09-03 21:16:00
4,B3,2018-09-03 21:05:00,2018-09-03 21:05:00
...,...,...,...
319068,D4,2016-06-05 17:25:00,2018-09-03 21:16:00
319069,E18,2015-07-09 13:38:00,2018-09-03 15:51:00
319070,E18,2015-07-09 13:38:00,2018-09-03 15:51:00
319071,E13,2016-05-31 19:35:00,2018-09-03 16:00:00


4 Pull all UserID, Name, ProgramName and FieldValue whether or not they have a FieldValue assigned to that program.

In [18]:
pd.set_option('display.max_columns', None)
combine_data.head(2)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,time_column,phase_of_day,MONTH_VALUE,CODE,NAME
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",13:00:00,Noon,September,619,LARCENY ALL OTHERS
1,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",13:00:00,Noon,September,619,LARCENY OTHER $200 & OVER


4 Pull all UserID, Name, ProgramName and FieldValue whether or not they have a FieldValue assigned to that program.
5 Pull ProgramName for ALL programs and the UserID for users assigned to those programs.

#### not able to Understand problem because in the data  we don't have This Field (UserID, Name, ProgramName and FieldValue)

### DATA WRANGLING EXERCISE :--->

1. Write a script that shows count of Auto Theft and Towed by Phase of Day(as index) vs Month(as column).
Phase of day is defined as:
Morning = 6 a.m to 11 a.m
Noon = 11.01 a.m to 5 p.m
Evening = 5.01 p.m to 8 p.m
Night = 8.01 p.m to 5.59 a.m
Sample Output
Jan Feb March.......Dec
Morning 1 2 3………………2
Noon 3 0 2………………3
Evening 1 0 0………………4
Night 0 1 1………………5


2. Write script to get offense (full offense name provided in offense_codes.csv file) per district which 
has maximum occurrence in respective district.

3. Add a column to data set which contains date of last incidents happened in respective district For Example: 
    if a state has N incidents, a column should be added to 
    dataset with (i+1)th incident having date of ith incident in that column
    
    
    
    
#### Above 3 question has already been achieved by the pandas dataframe 

In [21]:
!pip install pandasql

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=b91e6a21d5449878f0f3ae980f893d61d621bdf049622f3324fc047039df28f9
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\63\e8\ec\75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


In [22]:
from pandasql import sqldf


4) Write a script to identify street having maximum number of incidents for every district

In [28]:
# Define the SQL query
query = """
SELECT DISTRICT, COUNT(OFFENSE_DESCRIPTION) AS Offense_Count
FROM data
GROUP BY DISTRICT
"""

# Execute the SQL query
result = sqldf(query)
result

,DISTRICT,Offense_Count
0,None,1765
1,A1,35717
2,A15,6505
3,A7,13544
4,B2,49945
5,B3,35442
6,C11,42530
7,C6,23460
8,D14,20127
9,D4,41915


5) . Create a subset of data, with only 10 recent incidents for each Street

In [29]:
data.head(1)

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,time_column,phase_of_day,MONTH_VALUE,date_of_last_incidence_happened
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",13:00:00,Noon,September,2018-09-03 14:06:00


In [31]:
# Define SQL query
query = """
SELECT STREET, OCCURRED_ON_DATE
FROM (
    SELECT STREET, OCCURRED_ON_DATE,
           ROW_NUMBER() OVER (PARTITION BY Street ORDER BY OCCURRED_ON_DATE DESC) AS rn
    FROM data
) t
WHERE rn <= 10
"""

# Execute SQL query
subset = sqldf(query)

print(subset)

           STREET     OCCURRED_ON_DATE
0            None  2018-09-03 19:43:00
1            None  2018-09-03 18:20:00
2            None  2018-09-03 18:05:00
3            None  2018-09-03 18:05:00
4            None  2018-09-03 18:05:00
...           ...                  ...
31226  ZEIGLER ST  2018-06-05 12:37:00
31227  ZEIGLER ST  2018-06-04 07:29:00
31228  ZEIGLER ST  2018-05-28 22:23:00
31229   ZELLER ST  2018-02-22 13:42:00
31230   ZELLER ST  2016-05-16 15:45:00

[31231 rows x 2 columns]
